In [6]:
import numpy as np
import matplotlib.pyplot as plt
from keras import models,layers,optimizers,datasets,utils,losses,metrics
from keras import backend as K

In [7]:
batch_size = 16
latent_dim = 2  # Dimensionality of the latent space: a plane

# encoder model
input_img = layers.Input(shape=(28, 28, 1,))
x = layers.Conv2D(32, 3, padding='same', activation='relu')(input_img)
x = layers.Conv2D(64, 3, padding='same', activation='relu', strides=(2, 2))(x)
x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)
shape_before_flattening = K.int_shape(x)
x = layers.Flatten()(x)
x = layers.Dense(32, activation='relu')(x)

z_mean = layers.Dense(latent_dim)(x)
z_log_var = layers.Dense(latent_dim)(x)

In [8]:
def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim), mean=0., stddev=1.)
    return z_mean + K.exp(z_log_var) * epsilon

# generate input for decoder 
z = layers.Lambda(sampling)([z_mean, z_log_var])

In [9]:
# Decoder model
decoder_input = layers.Input(K.int_shape(z)[1:])
x = layers.Dense(np.prod(shape_before_flattening[1:]), activation='relu')(decoder_input)
x = layers.Reshape(shape_before_flattening[1:])(x)
x = layers.Conv2DTranspose(32, 3,
                           padding='same', activation='relu',
                           strides=(2, 2))(x)
x = layers.Conv2D(1, 3, padding='same', activation='sigmoid')(x)
decoder = models.Model(decoder_input, x)
output_img = decoder(z)

In [10]:
class CustomLossLayer(layers.Layer):

    def call(self, inputs):
        x = K.flatten(inputs[0])
        y = K.flatten(inputs[1])
        xent_loss = metrics.binary_crossentropy(x, y)
        kl_loss = -5e-4 * K.mean(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
        loss=K.mean(xent_loss + kl_loss)
        self.add_loss(loss, inputs=inputs)
        return x # return value not used
    
output = CustomLossLayer()([input_img, output_img])

In [11]:
from keras.datasets import mnist

(x_train, _), (x_test, _) = datasets.mnist.load_data()
x_train = x_train.reshape(60000, 28, 28, 1)/255
x_train = x_train[:6000,:,:,:]
x_test = x_test.reshape(10000, 28, 28, 1)/255
x_test = x_test[:1000,:,:,:]

vae = models.Model(input_img, output)
vae.compile(optimizer='rmsprop', loss=None)
vae.summary()

vae.fit(x=x_train, y=None,
        shuffle=True,
        epochs=10,
        batch_size=batch_size,
        validation_data=(x_test, None))

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
conv2d_5 (Conv2D)               (None, 28, 28, 32)   320         input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_6 (Conv2D)               (None, 14, 14, 64)   18496       conv2d_5[0][0]                   
__________________________________________________________________________________________________
conv2d_7 (Conv2D)               (None, 14, 14, 64)   36928       conv2d_6[0][0]                   
__________________________________________________________________________________________________
flatten_2 

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
from scipy.stats import norm

n = 15  # figure with 15x15 digits
digit_size = 28
figure = np.zeros((digit_size * n, digit_size * n))
# test latent variable in grid 
grid_x = norm.ppf(np.linspace(0.05, 0.95, n))
grid_y = norm.ppf(np.linspace(0.05, 0.95, n))

for i, yi in enumerate(grid_x):
    for j, xi in enumerate(grid_y):
        z_sample = np.array([[xi, yi]])
        z_sample = np.tile(z_sample, batch_size).reshape(batch_size, 2)
        x_decoded = decoder.predict(z_sample, batch_size=batch_size)
        digit = x_decoded[0].reshape(digit_size, digit_size)
        figure[i * digit_size: (i + 1) * digit_size,
               j * digit_size: (j + 1) * digit_size] = digit

plt.figure(figsize=(10, 10))
plt.imshow(figure, cmap='Greys_r')
plt.savefig('tmp.pdf', format='pdf')
plt.show()